In [2]:
import sys

In [9]:
sys.path.append('/home/long8v/BERT-NER')

In [10]:
from bert import Ner

In [12]:
model = Ner('/home/long8v/sroie_data/raw_bert_replace_digit_0921')

In [14]:
import pandas as pd

In [49]:
df = pd.read_csv('data/train.csv', sep=',', header=0)

In [62]:
to_string = lambda df : ' '.join(list(map(str, df[0].to_list())))

In [47]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/long8v/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [53]:
import pickle

In [56]:
with open('data/train_df.p', 'rb') as f:
    train_df = pickle.load(f)

In [64]:
string_list = list(map(lambda e: to_string(e), train_df))

In [73]:
train_df[5]

,0,1
0,-DOCSTART-,O
1,yong,B-COMPANY
2,ce,I-COMPANY
3,##n,I-COMPANY
4,enterprise,I-COMPANY
...,...,...
175,",",O
176,thank,O
177,you,O
178,.,O


In [23]:
from transformers import XLNetTokenizer, XLNetForTokenClassification
import tensorflow as tf

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetForTokenClassification.from_pretrained('xlnet-base-cased')

inputs = tokenizer("Hello, my dog is cute")
input_ids = inputs["input_ids"]
inputs["labels"] = tf.reshape(tf.constant([1] * tf.size(input_ids).numpy()), (-1, tf.size(input_ids))) 
# Batch size 1

outputs = model(inputs)
loss, scores = outputs[:2]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForTokenClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLNetForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForTokenClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: 

In [18]:
outputs

(<tf.Tensor: id=54363, shape=(9,), dtype=float32, numpy=
 array([2.5293472 , 1.4243901 , 1.91874   , 1.240661  , 1.4375057 ,
        1.0280666 , 0.87789565, 1.9522228 , 2.168625  ], dtype=float32)>,
 <tf.Tensor: id=54281, shape=(1, 9, 2), dtype=float32, numpy=
 array([[[7.9419885, 5.495709 ],
         [5.746722 , 4.5976315],
         [5.9655514, 4.205563 ],
         [5.93302  , 5.0337133],
         [4.4994793, 3.3331518],
         [5.0346904, 4.4493203],
         [4.080391 , 3.7397618],
         [4.7585106, 2.9593902],
         [4.4037113, 2.3565023]]], dtype=float32)>)

In [3]:

class Ner(TFXLNetForTokenClassification):

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None,valid_ids=None,attention_mask_label=None):
        sequence_output = self.bert(input_ids, token_type_ids, attention_mask,head_mask=None)[0]
        batch_size,max_len,feat_dim = sequence_output.shape
        valid_output = torch.zeros(batch_size,max_len,feat_dim,dtype=torch.float32,device='cuda')
        for i in range(batch_size):
            jj = -1
            for j in range(max_len):
                    if valid_ids[i][j].item() == 1:
                        jj += 1
                        valid_output[i][jj] = sequence_output[i][j]
        sequence_output = self.dropout(valid_output)
        logits = self.classifier(sequence_output)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=0)
            # Only keep active parts of the loss
            #attention_mask_label = None
            if attention_mask_label is not None:
                active_loss = attention_mask_label.view(-1) == 1
                active_logits = logits.view(-1, self.num_labels)[active_loss]
                active_labels = labels.view(-1)[active_loss]
                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        else:
            return logits


In [7]:
TFXLNetForTokenClassification()

TypeError: __init__() missing 1 required positional argument: 'config'

In [66]:
inference_result = list(map(lambda e: model.predict(e), string_list))

Token indices sequence length is longer than the specified maximum sequence length for this model (1057 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: CUDA error: device-side assert triggered

In [1]:
string_list[0]

NameError: name 'string_list' is not defined